In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# extract a line start with 'mse'
def extract_mse(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('mse'):
                return line
    return None

In [3]:
# extract the previous line of the last line start with 'Validation'
def extract_val(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('Validation'):
                line=lines[lines.index(line) - 1]
                float(re.findall(r"mse:\d+\.?\d*", line)[0][4:])
                return lines[lines.index(line) - 1]
    return None

In [4]:
# extract the float number behind 'mse:'
def extract_float(line):
    return float(re.findall(r"mse:\d+\.?\d*", line)[0][4:])

In [5]:
# find the last line start with 'Validation'
def extract_last_val(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('Validation'):
                line=float(re.findall(r"--> \d+\.?\d*", line)[0][4:])
                return line
    return None

In [6]:
# find the line start with 'FLOPs' and 'params', extract the following numbers
def extract_flops_params(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        flops=0
        params=0
        for line in lines:
            if line.startswith('FLOPs'):
                flops = float(re.findall(r"\d+\.?\d*", line)[0])
            if line.startswith('params'):
                params = float(re.findall(r"\d+\.?\d*", line)[0])
    return flops, params

In [7]:
# find the last line with 'Saving model ...' before the first line with '1 out of 3'
def extract_last_save(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        scaned=[]
        for line in lines:
            if line.startswith('Epoch:'):
                scaned.append(line)
            if line.startswith('EarlyStopping counter: 1 out of 3'):
                # return scaned[-3]
                # get the number after 'Test Loss: '
                return float(re.findall(r"Test Loss: \d+\.?\d*", scaned[-3])[0][10:])
    return None

In [8]:
import os
file_list = os.listdir('./')


In [9]:
file_list

['1j_FITS_Electricity_720_336_H4_bs64_s114.log',
 '1j_FITS_Electricity_720_720_H4_bs64_s114.log',
 '1j_FITS_Electricity_720_96_H4_bs64_s114.log',
 '1j_FITS_Electricity_720_192_H4_bs64_s114.log',
 '1j_FITS_Electricity_720_96_H6_bs64_s114.log',
 '1j_FITS_Electricity_720_336_H6_bs64_s114.log',
 '1j_FITS_Electricity_720_192_H6_bs64_s114.log',
 '1j_FITS_Electricity_720_720_H6_bs64_s114.log',
 '1j_FITS_Electricity_720_96_H8_bs64_s114.log',
 '1j_FITS_Electricity_720_192_H8_bs64_s114.log',
 '1j_FITS_Electricity_720_336_H8_bs64_s114.log',
 '1j_FITS_Electricity_720_720_H8_bs64_s114.log',
 '2j_FITS_Electricity_720_96_H6_bs64_s114.log',
 '2j_FITS_Electricity_720_720_H6_bs64_s114.log',
 '2j_FITS_Electricity_720_192_H6_bs64_s114.log',
 '2j_FITS_Electricity_720_336_H6_bs64_s114.log',
 '2j_FITS_Electricity_720_720_H8_bs64_s114.log',
 '2j_FITS_Electricity_720_192_H8_bs64_s114.log',
 '2j_FITS_Electricity_720_336_H8_bs64_s114.log',
 '2j_FITS_Electricity_720_96_H8_bs64_s114.log',
 '2j_FITS_Electricity_720

In [10]:
datasets = ['Electricity']
# datasets = ['weather', 'Exchange','Traffic','Electricity'] 'Ettm1','Ettm2',
train_mode = ['1','2']
Batchsize=['64']
H_order=['4','6','8','10']
output_len=['96','192','336','720']
input_len=['90','180','360','720']

In [11]:
writer=pd.ExcelWriter('./FITS_elec.xlsx')
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['1','2'],index=H_order)
            for mode in train_mode:
                for H in H_order:
                    file_name = mode+'j_FITS_'+dataset+'_'+input+'_'+output+'_H'+H+'_bs64_s114.log'
                    if file_name in file_list:
                        print(file_name)
                        line = extract_mse(file_name)
                        if line:
                            result=extract_float(line)
                            a.loc[H,mode]=result
                            # print(extract_last_val(file_name))
                        else:
                            print('None')
            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=3,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=6,startrow=startrow)
            elif input == '720':
                a.to_excel(writer,sheet_name=dataset,startcol=9,startrow=startrow)

writer.save()



1j_FITS_Electricity_90_96_H4_bs64_s114.log
1j_FITS_Electricity_90_96_H6_bs64_s114.log
1j_FITS_Electricity_90_96_H8_bs64_s114.log
2j_FITS_Electricity_90_96_H4_bs64_s114.log
2j_FITS_Electricity_90_96_H6_bs64_s114.log
2j_FITS_Electricity_90_96_H8_bs64_s114.log
1j_FITS_Electricity_180_96_H4_bs64_s114.log
1j_FITS_Electricity_180_96_H6_bs64_s114.log
1j_FITS_Electricity_180_96_H8_bs64_s114.log
2j_FITS_Electricity_180_96_H4_bs64_s114.log
2j_FITS_Electricity_180_96_H6_bs64_s114.log
2j_FITS_Electricity_180_96_H8_bs64_s114.log
1j_FITS_Electricity_360_96_H4_bs64_s114.log
1j_FITS_Electricity_360_96_H6_bs64_s114.log
1j_FITS_Electricity_360_96_H8_bs64_s114.log
2j_FITS_Electricity_360_96_H4_bs64_s114.log
2j_FITS_Electricity_360_96_H6_bs64_s114.log
2j_FITS_Electricity_360_96_H8_bs64_s114.log
1j_FITS_Electricity_720_96_H4_bs64_s114.log
1j_FITS_Electricity_720_96_H6_bs64_s114.log
1j_FITS_Electricity_720_96_H8_bs64_s114.log
2j_FITS_Electricity_720_96_H4_bs64_s114.log
2j_FITS_Electricity_720_96_H6_bs64_s11

2j_FITS_Electricity_720_192_H8_bs64_s114.log
1j_FITS_Electricity_90_336_H4_bs64_s114.log
1j_FITS_Electricity_90_336_H6_bs64_s114.log
1j_FITS_Electricity_90_336_H8_bs64_s114.log
2j_FITS_Electricity_90_336_H4_bs64_s114.log
2j_FITS_Electricity_90_336_H6_bs64_s114.log
2j_FITS_Electricity_90_336_H8_bs64_s114.log
1j_FITS_Electricity_180_336_H4_bs64_s114.log
1j_FITS_Electricity_180_336_H6_bs64_s114.log
1j_FITS_Electricity_180_336_H8_bs64_s114.log
2j_FITS_Electricity_180_336_H4_bs64_s114.log
2j_FITS_Electricity_180_336_H6_bs64_s114.log
2j_FITS_Electricity_180_336_H8_bs64_s114.log
1j_FITS_Electricity_360_336_H4_bs64_s114.log
1j_FITS_Electricity_360_336_H6_bs64_s114.log
1j_FITS_Electricity_360_336_H8_bs64_s114.log
2j_FITS_Electricity_360_336_H4_bs64_s114.log
2j_FITS_Electricity_360_336_H6_bs64_s114.log
2j_FITS_Electricity_360_336_H8_bs64_s114.log
1j_FITS_Electricity_720_336_H4_bs64_s114.log
1j_FITS_Electricity_720_336_H6_bs64_s114.log
1j_FITS_Electricity_720_336_H8_bs64_s114.log
2j_FITS_Electric

In [46]:
datasets = ['Ettm1','Ettm2']
# datasets = ['weather', 'Exchange','Traffic','Electricity'] ,
train_mode = ['1','2']
input_len=['90','180','360','720']
H_order=['4','6','8','10','12']
output_len=['96','192','336','720']

In [47]:
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['0','1','2'],index=H_order)
            for mode in train_mode:
                for H in H_order:
                    file_name = mode+'_FLinear_'+dataset+'_'+input+'_'+output+'_H'+H+'.log'
                    if file_name in file_list:
                        print(file_name)
                        line = extract_mse(file_name)
                        if line:
                            result=extract_float(line)
                            a.loc[H,mode]=result
                            # print(extract_last_val(file_name))
                        else:
                            print('None')
                        if mode=='2':
                            result0=extract_last_save(file_name)
                            a.loc[H,'0']=result0
            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=4,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=8,startrow=startrow)
            elif input == '720':
                a.to_excel(writer,sheet_name=dataset,startcol=12,startrow=startrow)

writer.save()

1_FLinear_Ettm1_90_96_H4.log
1_FLinear_Ettm1_90_96_H6.log
1_FLinear_Ettm1_90_96_H8.log
1_FLinear_Ettm1_90_96_H10.log
1_FLinear_Ettm1_90_96_H12.log
2_FLinear_Ettm1_90_96_H4.log
2_FLinear_Ettm1_90_96_H6.log
2_FLinear_Ettm1_90_96_H8.log
2_FLinear_Ettm1_90_96_H10.log
2_FLinear_Ettm1_90_96_H12.log
1_FLinear_Ettm1_180_96_H4.log
1_FLinear_Ettm1_180_96_H6.log
1_FLinear_Ettm1_180_96_H8.log
1_FLinear_Ettm1_180_96_H10.log
1_FLinear_Ettm1_180_96_H12.log
2_FLinear_Ettm1_180_96_H4.log
2_FLinear_Ettm1_180_96_H6.log
2_FLinear_Ettm1_180_96_H8.log
2_FLinear_Ettm1_180_96_H10.log
2_FLinear_Ettm1_180_96_H12.log
1_FLinear_Ettm1_360_96_H4.log
1_FLinear_Ettm1_360_96_H6.log
1_FLinear_Ettm1_360_96_H8.log
1_FLinear_Ettm1_360_96_H10.log
1_FLinear_Ettm1_360_96_H12.log
2_FLinear_Ettm1_360_96_H4.log
2_FLinear_Ettm1_360_96_H6.log
2_FLinear_Ettm1_360_96_H8.log
2_FLinear_Ettm1_360_96_H10.log
2_FLinear_Ettm1_360_96_H12.log
1_FLinear_Ettm1_720_96_H4.log
1_FLinear_Ettm1_720_96_H6.log
1_FLinear_Ettm1_720_96_H8.log
1_FLinea

In [12]:
writer=pd.ExcelWriter('./FLinear_flops.xlsx')
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['FLOPs','Params'],index=['4','6','8','10'])
            for freq in ['4','6','8','10']:
                file_name = '1j_FITS_Electricity_'+input+'_'+output+'_H'+freq+'_bs64_s114.log'
                if file_name in file_list:
                    print(file_name)
                    try:
                        flop, param = extract_flops_params(file_name)
                        a.loc[freq,'FLOPs']=flop
                        a.loc[freq,'Params']=param
                    except:
                        pass
                    
                else:
                    print('None')
                

            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=4,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=8,startrow=startrow)
            elif input == '720':
                a.to_excel(writer,sheet_name=dataset,startcol=12,startrow=startrow)

writer.save()

1j_FITS_Electricity_90_96_H4_bs64_s114.log
1j_FITS_Electricity_90_96_H6_bs64_s114.log
1j_FITS_Electricity_90_96_H8_bs64_s114.log
None
1j_FITS_Electricity_180_96_H4_bs64_s114.log
1j_FITS_Electricity_180_96_H6_bs64_s114.log
1j_FITS_Electricity_180_96_H8_bs64_s114.log
None
1j_FITS_Electricity_360_96_H4_bs64_s114.log
1j_FITS_Electricity_360_96_H6_bs64_s114.log
1j_FITS_Electricity_360_96_H8_bs64_s114.log
None
1j_FITS_Electricity_720_96_H4_bs64_s114.log
1j_FITS_Electricity_720_96_H6_bs64_s114.log
1j_FITS_Electricity_720_96_H8_bs64_s114.log
None
1j_FITS_Electricity_90_192_H4_bs64_s114.log
1j_FITS_Electricity_90_192_H6_bs64_s114.log
1j_FITS_Electricity_90_192_H8_bs64_s114.log
None
1j_FITS_Electricity_180_192_H4_bs64_s114.log
1j_FITS_Electricity_180_192_H6_bs64_s114.log
1j_FITS_Electricity_180_192_H8_bs64_s114.log
None
1j_FITS_Electricity_360_192_H4_bs64_s114.log
1j_FITS_Electricity_360_192_H6_bs64_s114.log
1j_FITS_Electricity_360_192_H8_bs64_s114.log
None
1j_FITS_Electricity_720_192_H4_bs64_s1


1j_FITS_Electricity_90_336_H6_bs64_s114.log
1j_FITS_Electricity_90_336_H8_bs64_s114.log
None
1j_FITS_Electricity_180_336_H4_bs64_s114.log
1j_FITS_Electricity_180_336_H6_bs64_s114.log
1j_FITS_Electricity_180_336_H8_bs64_s114.log
None
1j_FITS_Electricity_360_336_H4_bs64_s114.log
1j_FITS_Electricity_360_336_H6_bs64_s114.log
1j_FITS_Electricity_360_336_H8_bs64_s114.log
None
1j_FITS_Electricity_720_336_H4_bs64_s114.log
1j_FITS_Electricity_720_336_H6_bs64_s114.log
1j_FITS_Electricity_720_336_H8_bs64_s114.log
None
1j_FITS_Electricity_90_720_H4_bs64_s114.log
1j_FITS_Electricity_90_720_H6_bs64_s114.log
1j_FITS_Electricity_90_720_H8_bs64_s114.log
None
1j_FITS_Electricity_180_720_H4_bs64_s114.log
1j_FITS_Electricity_180_720_H6_bs64_s114.log
1j_FITS_Electricity_180_720_H8_bs64_s114.log
None
1j_FITS_Electricity_360_720_H4_bs64_s114.log
1j_FITS_Electricity_360_720_H6_bs64_s114.log
1j_FITS_Electricity_360_720_H8_bs64_s114.log
None
1j_FITS_Electricity_720_720_H4_bs64_s114.log
1j_FITS_Electricity_720_

In [11]:
datasets = ['Etth1']
# datasets = ['weather', 'Exchange','Traffic','Electricity'] 'Ettm1','Ettm2',
train_mode = ['1']
Batchsize=['64']
H_order=['6']
output_len=['96','192','336','720']
input_len=['720']

In [14]:
for output in output_len:
    mse=[]
    for seed in ['114', '514','1919','810','0']:
        file_name = '1_FITS_Etth1_360_'+output+'_H6_bs64_s'+seed+'.log'
        if file_name in file_list:
            print(file_name)
            line = extract_mse(file_name)
            if line:
                result=extract_float(line)
                # print(result)
                mse.append(result)
                # print(extract_last_val(file_name))
            else:
                print('None')
            # print(extract_last_val(file_name))
                
    print(np.mean(mse),np.std(mse))

1_FITS_Etth1_360_96_H6_bs64_s114.log
1_FITS_Etth1_360_96_H6_bs64_s514.log
1_FITS_Etth1_360_96_H6_bs64_s1919.log
1_FITS_Etth1_360_96_H6_bs64_s810.log
1_FITS_Etth1_360_96_H6_bs64_s0.log
0.3720731854438782 5.3644314739270755e-05
1_FITS_Etth1_360_192_H6_bs64_s114.log
1_FITS_Etth1_360_192_H6_bs64_s514.log
1_FITS_Etth1_360_192_H6_bs64_s1919.log
1_FITS_Etth1_360_192_H6_bs64_s810.log
1_FITS_Etth1_360_192_H6_bs64_s0.log
0.40439552068710327 2.6558648561293655e-05
1_FITS_Etth1_360_336_H6_bs64_s114.log
1_FITS_Etth1_360_336_H6_bs64_s514.log
1_FITS_Etth1_360_336_H6_bs64_s1919.log
1_FITS_Etth1_360_336_H6_bs64_s810.log
1_FITS_Etth1_360_336_H6_bs64_s0.log
0.4274980664253235 8.654898063138267e-05
1_FITS_Etth1_360_720_H6_bs64_s114.log
1_FITS_Etth1_360_720_H6_bs64_s514.log
1_FITS_Etth1_360_720_H6_bs64_s1919.log
1_FITS_Etth1_360_720_H6_bs64_s810.log
1_FITS_Etth1_360_720_H6_bs64_s0.log
0.4245747268199921 9.92559381291455e-05
